In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as spsp

## Load transactions data
For simplicity of prototype we only consider merchant groups

In [2]:
transactions_file = "../hackathon_data/avk_hackathon_data_transactions.csv"
useful_cols = ['party_rk', 'transaction_dttm', 'transaction_amt_rur', 'merchant_group_rk', 'category']
transactions = pd.read_csv(transactions_file, usecols=useful_cols, parse_dates=['transaction_dttm'])
print(transactions.shape)
transactions.head()

(11987617, 5)


,party_rk,transaction_dttm,transaction_amt_rur,merchant_group_rk,category
0,20337,2019-01-01,84.00,NaN,Сувениры
1,63404,2019-01-01,410.00,725.0,Фаст Фуд
2,24789,2019-01-01,701.44,NaN,Супермаркеты
3,57970,2019-01-01,6203.70,454.0,Дом/Ремонт
4,12232,2019-01-01,734.53,878.0,Супермаркеты


## Load socio-demographic data

In [3]:
socdem_file = "../hackathon_data/avk_hackathon_data_party_x_socdem.csv"
useful_cols = ['party_rk', 'age']
socdem = pd.read_csv(socdem_file, usecols=useful_cols)
print(socdem.shape)
socdem.head()

(50000, 2)


,party_rk,age
0,61243,70.0
1,66535,25.0
2,83721,55.0
3,88238,35.0
4,57179,30.0


## Merge datasets and select people aged 60+

In [4]:
df = pd.merge(transactions, socdem, on='party_rk').dropna(subset=['merchant_group_rk'])
df.drop(df[df.age < 60].index, inplace=True)
df.drop(columns='age', inplace=True)
print(df.shape)
df.head()

(344202, 5)


,party_rk,transaction_dttm,transaction_amt_rur,merchant_group_rk,category
19767,29815,2019-01-01,757.03,341.0,Супермаркеты
19775,29815,2019-01-01,314.00,341.0,Супермаркеты
19779,29815,2019-01-10,1470.61,341.0,Супермаркеты
19784,29815,2019-01-15,53.90,878.0,Супермаркеты
19792,29815,2019-01-17,251.00,589.0,Связь/Телеком


## Calculate monthly expenses for each merchant group

In [5]:
monthly = pd.Grouper(key='transaction_dttm',freq='M')
df_monthly = df.groupby([monthly, 'party_rk', 'merchant_group_rk', 'category']).agg({'transaction_amt_rur': 'sum'})
df_monthly.rename(columns={'transaction_amt_rur': 'monthly_amt'}, inplace=True)
print(df_monthly.shape)
df_monthly.head()

(133367, 1)


monthly_amt
transaction_dttm party_rk merchant_group_rk category                      
2019-01-31       8        205.0             Красота                3935.00
                 30       243.0             Финансовые услуги      5100.00
                          999.0             Супермаркеты            520.25
                          1211.0            Аптеки                 1257.00
                          2259.0            Финансовые услуги        11.00

## Find average monthly expenses for each merchant group

In [6]:
df_monthly = df_monthly.groupby(['party_rk', 'merchant_group_rk', 'category']).agg({'monthly_amt': 'mean'})
df_monthly.rename(columns={'monthly_amt': 'avg_monthly_amt'}, inplace=True)
df_monthly.reset_index(inplace=True)
print(df_monthly.shape)
df_monthly.head()

(53577, 4)


,party_rk,merchant_group_rk,category,avg_monthly_amt
0,8,100.0,Разные товары,311.0
1,8,113.0,Дом/Ремонт,69.0
2,8,205.0,Красота,3935.0
3,8,341.0,Супермаркеты,466.0
4,8,610.0,Супермаркеты,324.0


## Average monthly spendings over all merchant groups

In [7]:
df_monthly_overall = df_monthly.groupby(['party_rk']).agg({'avg_monthly_amt': 'sum'})
df_monthly_overall.rename(columns={'avg_monthly_amt': 'avg_monthly_amt_overall'}, inplace=True)
print(df_monthly_overall.shape)
df_monthly_overall.head()

(4141, 1)


,avg_monthly_amt_overall
party_rk,
8,7737.000000
11,13734.318333
15,248213.125833
30,9244.237143
32,30722.851667


## Proportion of money spent on each merchant group

In [8]:
df_monthly_portion = pd.merge(df_monthly, df_monthly_overall, on='party_rk')
df_monthly_portion['portion'] = df_monthly_portion.avg_monthly_amt / df_monthly_portion.avg_monthly_amt_overall
df_monthly_portion.drop(columns=['avg_monthly_amt', 'avg_monthly_amt_overall'], inplace=True)
print(df_monthly_portion.shape)
df_monthly_portion.head()

(53577, 4)


,party_rk,merchant_group_rk,category,portion
0,8,100.0,Разные товары,0.040196
1,8,113.0,Дом/Ремонт,0.008918
2,8,205.0,Красота,0.508595
3,8,341.0,Супермаркеты,0.060230
4,8,610.0,Супермаркеты,0.041877


### Check that sum of portions equals 1

In [9]:
portion_check = df_monthly_portion.groupby(['party_rk']).agg({'portion': 'sum'})
n_rows = portion_check.shape[0]
eps = 1e-12
n_ones = portion_check.loc[(portion_check.portion - 1).abs() < eps].shape[0]
assert n_rows == n_ones

## Renumber client IDs and merchant group IDs

In [10]:
print(f"Min client ID: {df_monthly_portion.party_rk.values.min()},", \
      f"max client ID: {df_monthly_portion.party_rk.values.max()}")
df_monthly_portion.party_rk = df_monthly_portion.party_rk.rank(method='dense').astype(int) - 1
print(f"Min client ID after renumbering: {df_monthly_portion.party_rk.values.min()},", \
      f"max client ID after renumbering: {df_monthly_portion.party_rk.values.max()}")

print(f"\nMin merchant group ID: {df_monthly_portion.merchant_group_rk.values.min()},", \
      f"max merchant group ID: {df_monthly_portion.merchant_group_rk.values.max()}")
df_monthly_portion.merchant_group_rk = df_monthly_portion.merchant_group_rk.rank(method='dense').astype(int) - 1
print(f"Min merchant group ID after renumbering: {df_monthly_portion.merchant_group_rk.values.min()},", \
      f"max merchant group ID after renumbering: {df_monthly_portion.merchant_group_rk.values.max()}")

Min client ID: 8, max client ID: 94521
Min client ID after renumbering: 0, max client ID after renumbering: 4140

Min merchant group ID: 3.0, max merchant group ID: 2957.0
Min merchant group ID after renumbering: 0, max merchant group ID after renumbering: 1774


## Save data
Data is saved as sparse matrix of shape **(number of clients, number of merchant groups)**. Elements of matrix are portions of money spend by customer on each merchant.

In [11]:
data_matrix = spsp.csr_matrix((df_monthly_portion.portion.values, 
                              (df_monthly_portion.party_rk.values, df_monthly_portion.merchant_group_rk.values)))
print(data_matrix.shape)
spsp.save_npz("data_matrix", data_matrix)

(4141, 1775)
